In [312]:
import pandas as pd
from sodapy import Socrata
from uszipcode import SearchEngine # note this does an approximation of zipcode based upon a radius
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

import pickle

In [313]:
pd.options.display.max_columns=100

In [314]:
client = Socrata("data.lacity.org", None)

results = client.get("7fvc-faax", limit=2000000)
# results = client.get("7fvc-faax", limit=200)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [315]:
results_df.head()

In [ ]:
# with open('results_df.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(results_df, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('results_df.pickle', 'rb') as f:
#     # The protocol version used is detected automatically, so we do not
#     # have to specify it.
#     results_df = pickle.load(f)

In [ ]:
list(results_df)

[':@computed_region_2dna_qi2s',
 ':@computed_region_k96s_3jcv',
 ':@computed_region_kqwf_mjcx',
 ':@computed_region_qz3q_ghft',
 ':@computed_region_tatf_ua23',
 ':@computed_region_ur2y_g4cx',
 'area_id',
 'area_name',
 'crm_cd',
 'crm_cd_1',
 'crm_cd_2',
 'crm_cd_3',
 'crm_cd_4',
 'crm_cd_desc',
 'cross_street',
 'date_occ',
 'date_rptd',
 'dr_no',
 'location',
 'location_1',
 'mocodes',
 'premis_cd',
 'premis_desc',
 'rpt_dist_no',
 'status',
 'status_desc',
 'time_occ',
 'vict_age',
 'vict_descent',
 'vict_sex',
 'weapon_desc',
 'weapon_used_cd']

In [ ]:
rows=len(results_df)
search = SearchEngine(simple_zipcode=True)
lon = []
lat = []
zipcode = []
year_single = '2000'
year = []
for i in range(rows):
    # single values for readability
    lattitude = float(results_df['location_1'][i]['coordinates'][1])
    longitude = float(results_df['location_1'][i]['coordinates'][0])
    year_single = results_df['date_occ'][i][:4]
    # append to arrays
    lon.append(longitude)
    lat.append(lattitude)
    try:
        zipcode.append(search.by_coordinates(lattitude, longitude, radius=5, returns=1)[0].zipcode)
    except:
        zipcode.append('00000')
        print(f'{i},{lattitude},{longitude}')
    year.append(year_single)
results_df['lat'] = lat
results_df['lon'] = lon
results_df['zipcode'] = zipcode
results_df['year'] = year

2513,0.0,0.0
12985,34.706,-118.0879
32768,0.0,0.0
40068,0.0,0.0
54647,0.0,0.0
142345,34.706,-118.0879
143901,0.0,0.0
143937,0.0,0.0
144892,0.0,0.0
144964,0.0,0.0
145194,0.0,0.0
145223,0.0,0.0
158157,0.0,0.0
181231,0.0,0.0
186642,34.576,-118.0115
194622,33.3427,-118.3258
202117,0.0,0.0
202226,0.0,0.0
202858,0.0,0.0
203448,0.0,0.0
203594,0.0,0.0
203820,0.0,0.0
203935,0.0,0.0
220141,0.0,0.0
241522,0.0,0.0
272819,0.0,0.0
274499,0.0,0.0
286349,0.0,0.0
321578,0.0,0.0
351718,34.6765,-118.1089
356908,0.0,0.0
361817,0.0,0.0
372580,0.0,0.0


In [ ]:
len(results_df)

In [ ]:
# remove lines with no corresponding zip code
results_df = results_df[results_df.zipcode != '00000']
len(results_df)

In [ ]:
# drop irrelevant columns
results_df.drop(columns=[':@computed_region_2dna_qi2s',
 ':@computed_region_k96s_3jcv',
 ':@computed_region_kqwf_mjcx',
 ':@computed_region_qz3q_ghft',
 ':@computed_region_tatf_ua23',
 ':@computed_region_ur2y_g4cx',
 'crm_cd_1',
 'crm_cd_2',
 'crm_cd_3',
 'cross_street'], inplace=True)

In [ ]:
results_df.head()

In [ ]:
file_name='los_angeles_crime_statistics.csv'
results_df.to_csv(file_name, encoding='utf-8')

In [ ]:
# dates

In [ ]:
# construct a basic timedelta and print it
print (timedelta(days=365, hours=0, minutes=0))
# print today's date
print ("today is: " + str(datetime.now()))
# print today's date one year from now
print ("one year from now it will be:" + str(datetime.now() - timedelta(days=365)))


In [ ]:
now = datetime.now()
print(now)
print(timedelta(days=365))
# 1year = datetime.now() - timedelta(days=365)
# 1year
print(type(now))

In [ ]:
df_0 = results_df

In [ ]:
# group by zipcode /crm_cd ; count number of crimes (dr_no) 
aggregation = { 
     'dr_no': 'nunique' 
}
df_1 = results_df.groupby(['zipcode','crm_cd']).agg(aggregation)
df_1

In [ ]:
df_1.plot(kind="line", figsize=(20,3))

In [ ]:
# group by year / zipcode ; count number of crimes (dr_no)
aggregation = { 
     'dr_no': 'nunique' 
}
df_2 = results_df.groupby(['year','zipcode']).agg(aggregation)
df_2

In [ ]:
fig = plt.figure()
ax = plt.axes()

x = np.linspace(0, 10, 1000)
ax.plot(x, np.sin(x));

In [ ]:
# group by year / zipcode ; count number of crimes (dr_no)
df_4 = results_df.sort_values(by=['zipcode'])
aggregation = { 
     'dr_no': 'count' 
}
df_3 = df_4.groupby(['zipcode']).agg(aggregation)
df_3 = df_3.sort_values(by=['dr_no'], ascending=False)
df_3 = df_3[:10]
df_3.reset_index(level=0, inplace=True)
df_3

In [ ]:
df_5 = df_3['zipcode']
df_5

In [ ]:
# rolling year

In [ ]:
# 5 years

In [ ]:
df_3 = df_3.sort_values(by='zipcode')
top_zipcodes = df_3.zipcode.unique()
top_zipcodes

In [ ]:
aggregation = { 
     'dr_no': 'nunique' 
}
zip_90012 = pd.DataFrame(df_0[df_0.zipcode == '90012'].groupby(['zipcode']).agg(aggregation))

# df_6.reset_index(level=0, inplace=True)
zip_90012

In [ ]:
plot('xlabel', 'ylabel', data=df_2)
plt.xlabel('Total Number of Rides (Per City)', fontdict=None, labelpad=None)
plt.ylabel('Average Fare ($)', fontdict=None, labelpad=None)
plt.title('Pyber Ride Sharing Data (2016)', fontdict=None, loc='center', pad=None)
plt.grid(b=None, which='major', axis='both')

In [ ]:
# income

In [ ]:
# home value

In [ ]:
result